In [1]:
from functions import *

In [4]:
# get pre-scraped text data
# evaluate information score only on relative articles

with open('rel_articles.txt', 'r') as file:
    data_code = file.read()

exec(data_code)  # Just to verify the content read


there are 8 articles in relevant_articles.
there are 8 articles in irrelevant_articles.


In [6]:
for article in relevant_articles:
    print(info_score(article))
    
# IMO MMSI Names (people/location)

2
2
2
2
2
2
2
2


#### Thought: For a search engine, it might be a good idea to add the information score for all relevant articles from the search.

#### Challenge: It is extremely rare an article directly contains IMO number or MMSI number. We need some more types of information to check!

In [29]:
########################################################
#### find_imo: Find IMO (7-digit numbers)

def find_imo(content):    
    # regular expression for a 7-digit number (exactly 7-digit)
    imo_pattern = r'\b\d{7}\b'
    
    # find all 7-digit numbers in the content 
    imo_list = re.findall(imo_pattern, content)
    
    return imo_list

########################################################
#### find_mmsi: Find MMSI (9-digit numbers)

def find_mmsi(content):    
    # regular expression for a 9-digit number (exactly 9-digit)
    mmsi_pattern = r'\b\d{9}\b'
    
    # find all 9-digit numbers in the content 
    mmsi_list = re.findall(mmsi_pattern, content)
    
    return mmsi_list

########################################################
#### find names (company, people, etc)
# These functions require spaCy
########################################################

import spacy

########################################################
# people's names
def find_people(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    names = []

    for ent in doc.ents:
        if ent.label_ in ['PERSON']: # NORP is about nationality/religious/political group
            names.append(ent.text)
    return names

########################################################
# company's names
def find_company(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    names = []

    for ent in doc.ents:
        if ent.label_ in ['ORG']: # NORP is about nationality/religious/political group
            names.append(ent.text)
    return names


########################################################
# country's name / location
def find_location(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    locations = []

    for ent in doc.ents:
        if ent.label_ in ['LOC', 'GPE', 'NORP']: # location labels
            locations.append(ent.text)
    return locations


########################################################
# fishing species


########################################################
# GRT


########################################################
# Information score
def weighted_info_score(text):
    #give weigths to each information:
    # IMO: 2
    # MMSI: 2
    # Involved parties or location: 1
    # The score will be out of 10
    score_list = [find_imo, find_mmsi, find_people, find_company, find_location]
    weight_vector = np.array([2, 2, 1, 1, 1])
    score = []
    for item in score_list:
        if len(item(text)) > 0:
            score.append(1)
        else:
            score.append(0)
    score = np.array(score)
    weighted_score = score * weight_vector
    #np.dot(score,weight_vector)

    return weighted_score



In [30]:
import numpy as np

text=  """
Hi here are some numbers 391923, 1341231, 35234123, 1412312, 3123. We are in Canada. Calgary is a big city. I have never been to Winnipeg. We have four awesome people in our group.
Sogol has been helped us as a mentor. I'm Sumin. My teammates are Clotilde, Irushi, and Hiva.
NCIS is the company! Apple has been launched a new phone. What about Fishing Corp.?
"""


weighted_info_score(text)

array([2, 0, 1, 1, 1])